In [30]:
import numpy as np
from bregman.suite import * 
import tensorflow as tf

In [31]:
k = 2
max_iterations = 100

In [32]:
filenames = tf.train.match_filenames_once('./audio_dataset/*.wav')
count_num_files = tf.size(filenames)
filename_queue = tf.train.string_input_producer(filenames)
reader = tf.WholeFileReader()
filename, file_contents = reader.read(filename_queue)
chroma = tf.placeholder(tf.float32)
max_freqs = tf.argmax(chroma, 0)

In [33]:
def get_next_chromagram(sess):
    audio_file = sess.run(filename)
    F = Chromagram(audio_file, nfft=16384, wfft=8192, nhop=2205)
    return F.X

In [34]:
def initial_cluster_centroids(X, k):
    return X[0:k, :]

In [50]:
def assign_cluster(X, centroids):
    expanded_vectors = tf.expand_dims(X, 0) # 1, 5, 12
    expanded_centroids = tf.expand_dims(centroids, 1) #2, 1, 12
    distances = tf.reduce_sum(tf.square(tf.subtract(expanded_vectors, expanded_centroids)), 2) #2, 5
    mins = tf.argmin(distances, 0)
    return mins

In [51]:
def recompute_centroids(X, Y):
    sums = tf.unsorted_segment_sum(X, Y, k)
    counts = tf.unsorted_segment_sum(tf.ones_like(X), Y, k)
    return sums / counts

In [52]:
def extract_feature_vector(sess, chroma_data):
    num_features, num_samples = np.shape(chroma_data)
    freq_vals = sess.run(max_freqs, feed_dict={chroma: chroma_data})
    hist, bins = np.histogram(freq_vals, bins=range(num_features + 1))
    return hist.astype(float) / num_samples

In [53]:
def get_dataset(sess):
    num_files = sess.run(count_num_files)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    xs = []
    for _ in range(num_files):
        chroma_data = get_next_chromagram(sess)
        x = [extract_feature_vector(sess, chroma_data)]
        x = np.matrix(x)
        if len(xs) == 0:
            xs = x
        else:
            xs = np.vstack((xs, x))
    return xs

In [54]:
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    X = get_dataset(sess)
    centroids = initial_cluster_centroids(X, k)
    i, converged = 0, False
    while not converged and i < max_iterations:
        i += 1
        Y = assign_cluster(X, centroids)
        centroids = sess.run(recompute_centroids(X, Y))
    print(centroids)

[[0.20750733 0.07575758 0.08406647 0.0259042  0.04363636 0.06848485
  0.16482893 0.10408602 0.05075269 0.06060606 0.08406647 0.03030303]
 [0.03533354 0.         0.         0.02040816 0.60234542 0.25982333
  0.         0.08208955 0.         0.         0.         0.        ]]
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[{{node input_producer/input_producer_EnqueueMany}}]]
